In [1]:
from tqdm import tqdm
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.spatial import distance
from heapq import nsmallest
from sklearn.metrics import pairwise_distances
import random

In [2]:
articles = pd.read_csv('articlesDataset/articlesTestDataset', delimiter=',', header=None)
tables = pd.read_csv('articlesDataset/cleanDataTables', delimiter=',', header=None)

In [3]:
formattedArticles = articles.iloc[:,:].values
formattedTables = tables.iloc[:,:].values

In [4]:
corpusTables = []

for tables in formattedTables:
    
    corpusTables.append(str(tables[1]))

In [5]:
vectorizer = TfidfVectorizer(stop_words='english', analyzer='word')
tablesByCharacteristics = vectorizer.fit_transform(corpusTables)

In [6]:
corpusArticlesID = []
corpusArticlesText = []

for articles in formattedArticles[0:1000]:
    
    articlePgID = articles[0]
    
    articleTitle = str(articles[1])
    
    articleText = str(articles[2])
    
    catchAll = articleTitle + ' ' + articleText
    
    corpusArticlesID.append(articlePgID)
        
    corpusArticlesText.append(str(articleTitle))  

In [7]:
def getIdRankedTalbes(topK,distanceVector):

    idRankedTables = []

    for topkDistance in topK:
        
        index = np.where(distanceVector == topkDistance)
         
        indexColummun = index[0][0]
        
        idRankedTables.append(formattedTables[indexColummun][0])

    return idRankedTables

In [8]:
def getAccuracy(idRankedTables, idQueryGoal):

    accuracy = 0

    for idTable in idRankedTables:
    
        if idTable == idQueryGoal:
    
            accuracy = 1
            break;

    return accuracy

In [9]:
def saveAccuracy(k,accuracy):
    
    if k == 1:
            
        AverageTop1.append(accuracy)
        
    if k == 10:
            
        AverageTop10.append(accuracy)
        
    if k == 100:
            
        AverageTop100.append(accuracy)
        
    if k == 1000:
            
        AverageTop1000.append(accuracy)

In [10]:
articlesByCharacteristics = vectorizer.transform(corpusArticlesText)

In [11]:
AverageTop1 = []
AverageTop10 = []
AverageTop100 = []
AverageTop1000 = []

topK = [1,10,100,1000]

for i in tqdm(range(len(corpusArticlesText)-990)):
    
    distanceVector = pairwise_distances(articlesByCharacteristics[i], tablesByCharacteristics, metric='cosine')
    
    idQueryGoal = int(corpusArticlesID[i])
    
    for accuracyK in topK:
        
        countTopTables = accuracyK
        
        topKRank = nsmallest(countTopTables, distanceVector[0])
    
        idRankedTables = getIdRankedTalbes(topKRank,distanceVector[0])
    
        accuracy_value = getAccuracy(idRankedTables,idQueryGoal)
        
        #save the accuracy on the list
        saveAccuracy(accuracyK,accuracy_value)

100%|██████████| 10/10 [00:18<00:00,  1.82s/it]


In [12]:
print(str(round(np.mean(AverageTop1),4))+" (±) "+str(round(np.std(AverageTop1),4)))
print(str(round(np.mean(AverageTop10),4))+" (±) "+str(round(np.std(AverageTop10),4)))
print(str(round(np.mean(AverageTop100),4))+" (±) "+str(round(np.std(AverageTop100),4)))
print(str(round(np.mean(AverageTop1000),4))+" (±) "+str(round(np.std(AverageTop1000),4)))

0.1 (±) 0.3
0.1 (±) 0.3
0.3 (±) 0.4583
0.4 (±) 0.4899
